In [1]:
!pip install kaggle

In [5]:
!makedir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

/bin/bash: line 1: makedir: command not found


In [6]:
#API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

 87% 70.0M/80.9M [00:00<00:00, 198MB/s]
100% 80.9M/80.9M [00:00<00:00, 189MB/s]


In [7]:
#extracting the compressed dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("dataset is extracted")

dataset is extracted


Importing the dependencies


In [9]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data processing


In [12]:
#loading the data from csv files to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding = 'ISO-8859-1')

In [13]:
#checking for the number of rows and columns
twitter_data.shape

(1599999, 6)

In [14]:
#printing the first five rows
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [15]:
#naming the columns and reading the dataset again
column_names = ['target','id','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names  = column_names,encoding = 'ISO-8859-1')

In [16]:
twitter_data.shape

(1600000, 6)

In [17]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [18]:
#counting the number of missing values in the dataset

twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [19]:
#checking the distribution of target columns
twitter_data['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [24]:
twitter_data.replace({'target' : {4:1}},inplace = True)

In [25]:
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

0 refers to negative tweet
1 refers to positive tweet


Stemming

In [26]:
port_stem = PorterStemmer()

In [27]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]',' ',content)

  stemmed_content = stemmed_content.lower()

  stemmed_content = stemmed_content.split()

  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]

  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [28]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [29]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [30]:
print(twitter_data["stemmed_content"])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [31]:
x = twitter_data['stemmed_content'].values
y = twitter_data['target'].values

In [32]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [33]:
print(y)

[0 0 0 ... 1 1 1]


In [34]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, stratify = y, random_state = 43 )

In [35]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [36]:
print(x_train)

['oh yeah total saw mr weston day' 'miss leonardo'
 'cupofstartup hello thank drop' ...
 'aschernathan love phoenix fave funki squared young ever feel better excel choic'
 'finish core synergist rough ey celtic suck egg' 'massiv headach go']


In [37]:
print(x_test)

['spencerpratt u go topshop' 'lol pay abort brennan'
 'bzmclaughlin quot twitter quot shall' ...
 'watch tri find kriswilliam bar scene' 'hilari grad parti senior'
 'clifsoulo aw hate becom quot quot kind girl happen']


In [38]:
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [39]:
print(x_train)

  (0, 93731)	0.20699650034124487
  (0, 439229)	0.6484094983441298
  (0, 275883)	0.3858153135027876
  (0, 354558)	0.33694882628783274
  (0, 413701)	0.3482910683569716
  (0, 453412)	0.2919836572179065
  (0, 299191)	0.2603237969414325
  (1, 232092)	0.9412228152060049
  (1, 267640)	0.3377863409548747
  (2, 109532)	0.39623345684257455
  (2, 400383)	0.2412116254753622
  (2, 163497)	0.37204901144441266
  (2, 86533)	0.8039872713528196
  (3, 138659)	0.5851508213432324
  (3, 406634)	0.36256371221123757
  (3, 375719)	0.23676705977759777
  (3, 328768)	0.5851508213432324
  (3, 245525)	0.23440219436220727
  (3, 42963)	0.21105108226717187
  (3, 171147)	0.16792557430777727
  (4, 136342)	0.38204696263245547
  (4, 31081)	0.20074762457315343
  (4, 5600)	0.6044439968890563
  (4, 454096)	0.33532883868175245
  (4, 308177)	0.5796054987184656
  :	:
  (1279996, 170252)	0.2599704951468356
  (1279996, 453349)	0.2453450511763687
  (1279997, 379979)	0.47904605396796335
  (1279997, 24329)	0.41417264375923896
  (127

In [40]:
print(x_test)

  (0, 413294)	0.6816628920368518
  (0, 378475)	0.6954374135378855
  (0, 149717)	0.22738184947724233
  (1, 310736)	0.3742878186196457
  (1, 239668)	0.24136930892768313
  (1, 51393)	0.6629013231404643
  (1, 1938)	0.6018399465965529
  (2, 420731)	0.3648880609011842
  (2, 361499)	0.5762524573222674
  (2, 329759)	0.7312932438097743
  (3, 441082)	0.2534118967460602
  (3, 435798)	0.5368287855322745
  (3, 373408)	0.4658670272087246
  (3, 83594)	0.43629814267722355
  (3, 80409)	0.3107550361696134
  (3, 49782)	0.37899923055705553
  (4, 349636)	0.22049249725496994
  (4, 258532)	0.3965032992557808
  (4, 249069)	0.2037743444410587
  (4, 248664)	0.3318489349435939
  (4, 222469)	0.4389464303511415
  (4, 80622)	0.3415970928439631
  (4, 10911)	0.5771983633903601
  (5, 444599)	0.3244400353707334
  (5, 438123)	0.2906339983240473
  :	:
  (319994, 241922)	0.20418223507215358
  (319994, 234585)	0.20294474831724843
  (319994, 78706)	0.22506751693561536
  (319995, 331762)	1.0
  (319996, 411208)	0.294830714830

In [41]:
model = LogisticRegression(max_iter = 1000)

In [42]:
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

In [43]:
x_train_prediction = model.predict(x_train)

training_data_accuracy = accuracy_score(y_train,x_train_prediction)

In [44]:
print("Accuracy score on the training data : ",training_data_accuracy)

Accuracy score on the training data :  0.81025859375


In [45]:
x_test_prediction = model.predict(x_test)

test_data_accuracy = accuracy_score(y_test,x_test_prediction)

In [46]:
print("Accuracy score on the training data : ",test_data_accuracy)

Accuracy score on the training data :  0.7780625


In [47]:
import pickle

In [48]:
filename = "trained_model.sav"
pickle.dump(model,open(filename,'wb'))

In [49]:
loaded_model = pickle.load(open('/content/trained_model.sav','rb'))

In [50]:
x_new =x_test[200]

print(y_test[200])

prediction = model.predict(x_new)

print(prediction)

if (prediction[0] == 0) :
  print('Negative Tweet')


else:
  print('positive tweet')

0
[0]
Negative Tweet


In [52]:
x_new =x_test[4]

print(y_test[4])

prediction = model.predict(x_new)

print(prediction)

if (prediction[0] == 0) :
  print('Negative Tweet')


else:
  print('positive tweet')

0
[0]
Negative Tweet
